# Demystifying AWS SageMaker Training for scikit-learn Lovers 
> How to build, train, and deploy a machine learning model with Amazon SageMaker .

- toc: true 
- badges: true
- comments: true
- categories: [aws, ml, sagemaker]
- keyword: [aws, ml, sagemaker]
- image: images/copied_from_nb/images/2022-06-08-sagemaker-training-overview.jpeg

![](images/2022-06-08-sagemaker-training-overview.jpeg)

# Enviornment

This notebook is prepared with Amazon SageMaker Studio using `Python 3 (Data Science)` Kernel and `ml.t3.medium` instance.

# About

This post is about simplifying Amazon SageMaker end-to-end machine learning workflow for newcomers. We will go slowly and try to understand each step involved in training and deploying a model. As an example, we will take [Kaggle Boston Housing dataset](https://www.kaggle.com/c/boston-housing) and train a SageMaker built-in algorithm [Linear Learner](https://docs.aws.amazon.com/sagemaker/latest/dg/linear-learner.html) on it. At the end of this tutorial, you will know how to use Amazon SageMaker to build, train, and deploy a machine learning model.

# Introduction

A typical SageMaker machine learning flow has the following five steps. If you have a good understanding of them then you can use this approach to train any other model with SageMaker.
1. **Put data on S3 bucket**<br>
   In most use cases, you will keep your training data on S3 bucket. You may also require to preprocess your data and for this you can use [SageMaker Data Wrangler](https://hassaanbinaslam.github.io/myblog/aws/ml/sagemaker/2022/05/17/aws-sagemaker-wrangler-p1.html). In this post, we will consider that data has already been processed and will upload it to S3 for training.
2. **Configure the training job**<br>
   While configuring a training job you need to take care of the following requirements <br>
      a. select the algorithm you want to use for your training <br>
      b. set the hyperparameters (if any) <br>
      c. define the infrastructure requirements like how many CPUs or GPUs you want to throw at your training run
3. **Launch training job**<br>
   Here we will tell the training job where the input data is located, and where should the output artifacts be stored once the training is done. With this setting, we are ready to start the training run. Once the training is started SageMaker will automatically provision the required infrastructure for our run, and once the training is complete it will be terminated, and you will only be billed for what you have used.
4. **Deploy model and make predictions**<br>
   Deploy the model to make real-time predictions. Again, you need to define the infrastructure requirements where you want your model to be deployed.
5. **Clean Up (Optional)**<br>
   If you are experimenting, you may want to terminate the machine on which you have deployed your model to avoid any unnecessary charges.

# Put Data on S3 Bucket

## Reading and Checking the Data

In this post, we will be using [Boston Housing Dataset](https://www.kaggle.com/c/boston-housing). This is a small dataset with 506 rows and 14 columns. `medv` is the target variable which means `median value of owner-occupied homes in $1000s`. This dataset is also available with this notebook. Let's read it and see how it looks.

In [3]:
import pandas as pd
import numpy as np

data_location = "./datasets/2022-06-08-sagemaker-training-overview/"

df = pd.read_csv(data_location + 'housing.csv')
df.head()

,crim,zn,indus,chas,nox,age,rm,dis,rad,tax,ptratio,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,5.33,36.2


let's quickly check the dimensions of our loaded dataset. 

In [4]:
df.shape

(506, 13)

The good thing about this dataset is that it does not require any preprocessing as all the features are already in numerical format (no categorical features), and also there are no missing values. We can quickly verify these assumptions.

Check the feature data types.

In [5]:
df.dtypes

crim       float64
zn         float64
indus      float64
chas         int64
nox        float64
age        float64
rm         float64
dis        float64
rad          int64
tax        float64
ptratio    float64
lstat      float64
medv       float64
dtype: object

Check if there are any missing values in our dataset.

In [6]:
df.isnull().values.any()

False

## Preparing the Data

At this point, our data is ready to be used for training but we also need to check the algorithm we want to use for any specific requirements. We have selected Linear Learner so let's check its documentation: [Linear Learner Algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/linear-learner.html)

In the documentation it says 
>For training, the linear learner algorithm supports both recordIO-wrapped protobuf and CSV formats. For the application/x-recordio-protobuf input type, only Float32 tensors are supported. For the text/csv input type, the first column is assumed to be the label, which is the target variable for prediction. You can use either File mode or Pipe mode to train linear learner models on data that is formatted as recordIO-wrapped-protobuf or as CSV.

It means that we can use CSV format for our training data. It also mentions that the first column in the training dataset should be the target label. So let's move our target label `medv` to the first column.

In [7]:
df = pd.concat([df['medv'], df.drop('medv', axis='columns')], axis='columns')

df.head()

,medv,crim,zn,indus,chas,nox,age,rm,dis,rad,tax,ptratio,lstat
0,24.0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,4.98
1,21.6,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,9.14
2,34.7,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,4.03
3,33.4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,2.94
4,36.2,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,5.33


We can now proceed with splitting our data into train and validation sets. After splitting we will export it as CSV files so it could be uploaded to S3 in the next section. Before exporting our dataframe into any format we should also check SageMaker general instructions for that format. For this use the link [Common Data Formats for Training](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-training.html) and check section **Using CSV Format**. Here it mentions

>To use data in CSV format for training, in the input data channel specification, specify text/csv as the ContentType. Amazon SageMaker requires that a CSV file does not have a header record and that the target variable is in the first column.

We have already moved our target label to the first column, and while exporting our dataframe to CSV format we should omit column headers as well.

In [8]:
from sklearn.model_selection import train_test_split

# out data size is very small so we will use a small test set
training_data, validation_data = train_test_split(df, test_size=0.1, random_state=42)

training_data.to_csv(data_location + "training_data.csv", index=False, header=False)
validation_data.to_csv(data_location + "validation_data.csv", index=False, header=False)

The next step is to upload this data to S3 bucket, and for this we will take the help of SageMaker Python SDK. There are two Python SDKs (Software Development Kit) available for SageMaker.

1. **AWS SDK for Python (Boto3)**. It provides low-level access to SageMaker APIs.
2. **SageMaker Python SDK**. It provides a high-level API interface, and you can do more with fewer lines of code. Internally it is calling Boto3 APIs.

We will be using **SageMaker Python SDK** for this post, and you will see that it has an interface similar to `scikit-learn`, and is a more natural choice for Data Scientists. SageMaker Python SDK documentation is super helpful, and it provides many examples to understand the working of its interface. Make sure that you check it out as well [sagemaker.readthedocs.io](https://sagemaker.readthedocs.io/en/stable/). If you don't have much time then I would suggest at least read the following sections from the documentation as we will be using them in the coming sections.

* [Initialize a SageMaker Session](https://sagemaker.readthedocs.io/en/stable/api/utility/session.html#sagemaker.session.Session)
* [Upload local file or directory to S3](https://sagemaker.readthedocs.io/en/stable/api/utility/session.html#sagemaker.session.Session.upload_data)
* [default_bucket](https://sagemaker.readthedocs.io/en/stable/api/utility/session.html#sagemaker.session.Session.default_bucket)
* [Create an Amazon SageMaker training job](https://sagemaker.readthedocs.io/en/stable/api/utility/session.html#sagemaker.session.Session.train)
* [A generic Estimator to train using any supplied algorithm](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Estimator)

Since we are already running this notebook from SageMaker environment, we don't need to care about credentials and permissions. We can simply start our new session with SageMaker environment using its SDK.

In [9]:
import sagemaker

session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = session.default_bucket()
region = session.boto_region_name

print(f"sagemaker.__version__: {sagemaker.__version__}")
print(f"Session: {session}")
print(f"Role: {role}")
print(f"Bucket: {bucket}")
print(f"Region: {region}")

sagemaker.__version__: 2.88.1
Session: <sagemaker.session.Session object at 0x7ff91637cd50>
Role: arn:aws:iam::801598032724:role/service-role/AmazonSageMaker-ExecutionRole-20220516T161743
Bucket: sagemaker-us-east-1-801598032724
Region: us-east-1


What we have done is
* imported the SageMaker Python SDK into our runtime
* get a session to work with SageMaker API and other AWS services
* get the execution role associated with the user profile. It is the same profile that is available to the user to work from console UI and has `AmazonSageMakerFullAccess` policy attached to it.
* create or get a default bucket to use and return its name. Default bucket name has the format `sagemaker-{region}-{account_id}`. If it doesn't exist then our session will automatically create it. You may also use any other bucket in its place given that you have enough permission for reading and writing.
* get the region name attached to our session

Next, we will use this session to upload data to our default bucket. 

In [10]:
##
# You may choose any other prefix for your bucket. All the data related to this post will be under this prefix.
bucket_prefix = '2022-06-08-sagemaker-training-overview'

Let's upload our training data first. In the output, we will get the complete path (S3 URI) for our uploaded data.

In [11]:
s3_train_data_path = session.upload_data(
    path=data_location + "training_data.csv",
    bucket=bucket,
    key_prefix=bucket_prefix + '/input/training'
)

print(s3_train_data_path)

s3://sagemaker-us-east-1-801598032724/2022-06-08-sagemaker-training-overview/input/training/training_data.csv


Let's do the same for our validation data.

In [12]:
s3_validation_data_path =  session.upload_data(
    path=data_location + "validation_data.csv",
    bucket=bucket,
    key_prefix=bucket_prefix + '/input/validation_data'
)

print(s3_validation_data_path)

s3://sagemaker-us-east-1-801598032724/2022-06-08-sagemaker-training-overview/input/validation_data/validation_data.csv


At this point we have our data available on S3 bucket. We can now proceed to the next step and configure our training job.

# Configure the Training Job

In this section, we will first retrieve the Docker container that is relevant to our training algorithm. Then we will create "sagemaker.estimator.Estimator" class object. This `estimator` object provides a high-level API interface to control end-to-end SageMaker training and deployment tasks. From the estimator, we will also define our infrastructure and hyperparameter tuning requirements. So let's get started.

## Finding the Right Docker Container

AWS SageMaker built-in algorithms are fully managed containers that can be accessed with one call. Each algorithm has a separate container and is also dependent on the region in which you want to run it. Getting the container URI is not a problem as long as we know about the region and the algorithm framework name. We already have the region name from our SageMaker session. To get the algorithm framework name (for linear learner) visit the [AWS Docker Registry Paths page](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html). From this page select your region. In my case it is `us-east-1`. On the [regional docker registry page](https://docs.aws.amazon.com/sagemaker/latest/dg/ecr-us-east-1.html) find the algorithm you want to use; `Linear Learner` in our case. This will give you the example code and algorithm framework name as shown below.

![linear-learner-framework-name](images/2022-06-08-sagemaker-training-overview/linear-learner-framework-name.png)

So let's use the provided sample code to get the container URI for our linear learner algorithm.

In [13]:
from sagemaker import image_uris

image_uri = image_uris.retrieve(framework='linear-learner',region=region)

print(image_uri)

382416733822.dkr.ecr.us-east-1.amazonaws.com/linear-learner:1


## Configure the Estimator Object

For configuring our estimator we need to fulfill the following requirements.
* **output path** Define the output path where we want to store the trained model artifacts
* **instance type** Since we have a small dataset and not so complex model so a small machine should suffice. 'ml.m5.large' will do. It is a general purpose instance with 2vCPU and 8GiB RAM. 
* **hyperparameters** For the hyperparameters check the [Linear Learner model documentaion](https://docs.aws.amazon.com/sagemaker/latest/dg/ll_hyperparameters.html). From the documentation, we find that the most important parameters for our problem are
   * `predictor_type`: which should be 'regressor' in our case
   * `mini_batch_size`: default is 1000 which is too large for our small dataset. Let's use 30 instead.

It is also important to note that the Estimator class will automatically provision a separate `ml.m5.large` machine to start the training run. This machine will be different from the one on which we are running this Jupyter notebook. Once training is complete this new machine will be terminated and we will be billed for only the time we have used it. This approach makes SageMaker very cost effective. We can keep using small less powerful machines for running Jupyter notebooks, and for training and other heavy workloads, we can provision separate machines for short durations and avoid any unnecessary bills.

In [14]:
##
# define the output path to store trained model artifacts
s3_output_path = f"s3://{bucket}/{bucket_prefix}/output/"

print(s3_output_path)

s3://sagemaker-us-east-1-801598032724/2022-06-08-sagemaker-training-overview/output/


In [15]:
from sagemaker.estimator import Estimator

ll_estimator = Estimator(
    image_uri = image_uri, # algorithm container
    role = role, # execution role with necessary permissions
    instance_count = 1,
    instance_type = 'ml.m5.large',
    sagemaker_session = session, # SageMaker API session
    output_path = s3_output_path, # training artifacts output path
    hyperparameters = {
        'predictor_type': 'regressor',
        'mini_batch_size': 30
    }
)

In the above cell, we have defined the hyperparameters within the `Estimator` object constructor. There is a second way to pass the hyperparameters to the Estimator object using the 'set_hyperparameters' function call. This method can be useful when we have a large number of parameters to set, or when we want to change them in multiple training runs.

```python
ll_estimator.set_hyperparameters(
    predictor_type='regressor', 
    mini_batch_size=30)
```

You might ask that for our problem even a small `ml.t3.medium` or `ml.c5.large` machine would have been sufficient. Why have not we used them? The answer to this is that AWS SageMaker at this time supports a limited number of machine types for training jobs and both of them are not supported. If you configure the Estimator object for these instance types you will get an error shown below

```
An error occurred (ValidationException) when calling the CreateTrainingJob operation: 1 validation error detected: Value 'ml.t3.medium' at 'resourceConfig.instanceType' failed to satisfy constraint: Member must satisfy enum value set: 

[ml.p2.xlarge, ml.m5.4xlarge, ml.m4.16xlarge, ml.p4d.24xlarge, ml.g5.2xlarge, ml.c5n.xlarge, ml.p3.16xlarge, ml.m5.large, ml.p2.16xlarge, ml.g5.4xlarge, ml.c4.2xlarge, ml.c5.2xlarge, ml.c4.4xlarge, ml.g5.8xlarge, ml.c5.4xlarge, ml.c5n.18xlarge, ml.g4dn.xlarge, ml.g4dn.12xlarge, ml.c4.8xlarge, ml.g4dn.2xlarge, ml.c5.9xlarge, ml.g4dn.4xlarge, ml.c5.xlarge, ml.g4dn.16xlarge, ml.c4.xlarge, ml.g4dn.8xlarge, ml.g5.xlarge, ml.c5n.2xlarge, ml.g5.12xlarge, ml.g5.24xlarge, ml.c5n.4xlarge, ml.c5.18xlarge, ml.p3dn.24xlarge, ml.g5.48xlarge, ml.g5.16xlarge, ml.p3.2xlarge, ml.m5.xlarge, ml.m4.10xlarge, ml.c5n.9xlarge, ml.m5.12xlarge, ml.m4.xlarge, ml.m5.24xlarge, ml.m4.2xlarge, ml.p2.8xlarge, ml.m5.2xlarge, ml.p3.8xlarge, ml.m4.4xlarge]
```

# Launch Training Job

To start our training run we need to associate our Estimator object with the training data. Our data is available on S3 bucket but we also need to tell our Estimator object in which format this data is provided. Is it in CSV format? Is it compressed or not? For these data format issues, we can use `Input Channels`. An input channel is configurations for S3 and file system data sources for AWS SageMaker ([check the input docs](https://sagemaker.readthedocs.io/en/stable/api/utility/inputs.html)). All SageMaker built-in algorithms require at least one training input channel, and more can be passed for validation and testing. In our case, we have two channels and both are in CSV format. Let's configure them.

In [16]:
from sagemaker.session import TrainingInput

train_input = TrainingInput(s3_train_data_path, content_type="text/csv")
validation_input = TrainingInput(s3_validation_data_path, content_type="text/csv")


ll_data = {
    'train': train_input,
    'validation': validation_input
}

Make sure that you use content_type `text/csv`. Only providing `csv` will not work and you will get the exception
```
Error for Training job linear-learner-2022-06-15-07-58-01-908: Failed. Reason: ClientError: No iterator has been registered for ContentType ('csv', '1.0'), exit code: 2
```

Alright, we are ready. To start the training run call estimator `fit` function and pass the data input channels. You can read more about `fit` call from docs [sagemaker.estimator.Estimator.fit](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Estimator.fit)

In [17]:
#collapse-output
ll_estimator.fit(ll_data)

2022-06-16 09:04:57 Starting - Starting the training job...
2022-06-16 09:05:23 Starting - Preparing the instances for trainingProfilerReport-1655370297: InProgress
.........
2022-06-16 09:06:48 Downloading - Downloading input data......
2022-06-16 09:07:44 Training - Downloading the training image...
2022-06-16 09:08:24 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[06/16/2022 09:08:32 INFO 139637139089216] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bia

Great! The training is done but it has also generated a lot of logs in the output. These logs are also available in the CloudWatch and if you want you can disable them from the `fit` function using `logs='None'` parameter. Let's try to analyze what information is presented in these logs.

The first part of these logs is related to infrastructure provisioning, downloading the training container, the data, and starting the training run.
```
2022-06-15 10:19:35 Starting - Starting the training job...
2022-06-15 10:19:51 Starting - Preparing the instances for trainingProfilerReport-1655288375: InProgress ......
2022-06-15 10:21:04 Downloading - Downloading input data......
2022-06-15 10:21:50 Training - Downloading the training image...
2022-06-15 10:22:35 Training - Training image download completed. Training in progress...
```
The second part of these logs is related to training performance metrics. The majority of the logs belong to this part.
```
[06/15/2022 10:22:53 INFO 140271381174080] #validation_score (algo-1) : ('mse_objective', 12.65540687710631)
[06/15/2022 10:22:53 INFO 140271381174080] #validation_score (algo-1) : ('mse', 12.65540687710631)
[06/15/2022 10:22:53 INFO 140271381174080] #validation_score (algo-1) : ('absolute_loss', 2.5288796518363204)
[06/15/2022 10:22:53 INFO 140271381174080] #validation_score (algo-1) : ('rmse', 3.557443868440697)
[06/15/2022 10:22:53 INFO 140271381174080] #validation_score (algo-1) : ('r2', 0.7973001536506935)
[06/15/2022 10:22:53 INFO 140271381174080] #validation_score (algo-1) : ('mae', 2.5288796144373276)
```

The third and final part is telling us about the Training job status and billable time in seconds.
```
2022-06-16 09:08:50 Uploading - Uploading generated training model
2022-06-16 09:09:24 Completed - Training job completed
Training seconds: 132
Billable seconds: 132
```
In our case, the billable seconds are 132. This billable time is for the `ml.m5.large` instance that we have configured for our training run. To find the billable amount we first need to find the price rate for our selected machine. For this go to the SageMaker pricing link [https://aws.amazon.com/sagemaker/pricing/](https://aws.amazon.com/sagemaker/pricing/) and select `On Demand Pricing`. From the given tabs click on the **Training tab**. Select your region `US East (N. Virginia)`. This will show you the pricing of different training instances. This page can also be used to find the available training instance types in your region. From this page, we can find that the price for our instance type `ml.m5.large` is `$0.115`. This price rate is per hour so we need to convert our billable time to per hour before multiplying it. i.e. (132/3600) * 0.115 = $0.0042 which is less than a penny for our training job.

We can also use Estimator object to get important information related to our training run. 

In [18]:
##
# to get the output path where the trained model artifacts are stored
ll_estimator.output_path

's3://sagemaker-us-east-1-801598032724/2022-06-08-sagemaker-training-overview/output/'

In [19]:
##
# to get the training job name
ll_estimator.latest_training_job.job_name

'linear-learner-2022-06-16-09-04-57-576'

In [20]:
##
# to get the trained model location
ll_estimator.model_data

's3://sagemaker-us-east-1-801598032724/2022-06-08-sagemaker-training-overview/output/linear-learner-2022-06-16-09-04-57-576/output/model.tar.gz'

In [21]:
#collapse-output
# to get all the configuration information for our training run
ll_estimator.latest_training_job.describe()

{'TrainingJobName': 'linear-learner-2022-06-16-09-04-57-576',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:801598032724:training-job/linear-learner-2022-06-16-09-04-57-576',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-us-east-1-801598032724/2022-06-08-sagemaker-training-overview/output/linear-learner-2022-06-16-09-04-57-576/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'mini_batch_size': '30', 'predictor_type': 'regressor'},
 'AlgorithmSpecification': {'TrainingImage': '382416733822.dkr.ecr.us-east-1.amazonaws.com/linear-learner:1',
  'TrainingInputMode': 'File',
  'MetricDefinitions': [{'Name': 'train:progress',
    'Regex': '#progress_metric: host=\\S+, completed (\\S+) %'},
   {'Name': 'validation:mae',
    'Regex': '#quality_metric: host=\\S+, validation mae <loss>=(\\S+)'},
   {'Name': 'train:objective_loss',
    'Regex': '#quality_metric: host=\\S+, epoch=\\S+, train \\S+_objective <loss>=(\\S+)'},
 

Training logs can also be viewed on AWS CloudWatch under LogGroup `/aws/sagemaker/TrainingJobs` and Log Stream same as the training job name.

# Deploy Model and Make Predictions

Our model is trained and now we can deploy it to AWS SageMaker endpoint with a single call. These endpoints are fully managed and support autoscaling.

In the deployment call we are required to provide
* **instance type** this is the machine type on which we want to deploy our model
* **instance count** number of instances we want to provision for our machine. If more than one instance is provisioned then SageMaker EndPoint will automatically load balance between them
* **endpoint_name** a unique identifier for your endpoint. If it is not provided then it will default to your training job name.

It is important to note that endpoints are scoped to an individual AWS account, and are not public. The URL does not contain the account ID, but SageMaker determines the account ID from the authentication token that is supplied by the caller. We will discuss in a separate post on how to use AWS Lambda and API Gateway to set up and deploy your inference endpoint for public access.

Read more about deployment from the documentation [sagemaker.estimator.Estimator.deploy](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Estimator.deploy)

In [27]:
##
# define the endpoint. can be any unique string
endpoint_name = ll_estimator.latest_training_job.job_name

print(endpoint_name)

linear-learner-2022-06-16-09-04-57-576


In [23]:
##
# deploy the model
ll_predictor = ll_estimator.deploy(
    initial_instance_count=1, 
    instance_type='ml.t2.medium',
    endpoint_name=endpoint_name
)

-------------!

This call will take a minute or so to spin up the required machine and create the endpoint. Once it is ready it will be available in **SageMaker Resources / Endpoints**.

![model-endpoint-sagemaker](images/2022-06-08-sagemaker-training-overview/model-endpoint-sagemaker.PNG)

In [24]:
##
# a test sample to get live inference from our model
test_sample = '0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,4.98'

Similar to data input channels that tell the Estimator object on how to read the data from S3 bucket, we need to tell our `predictor` object on how to receive (encode) input and return (decode) data during inference. For this we will use [sagemaker.serializers.CSVSerializer](https://sagemaker.readthedocs.io/en/stable/api/inference/serializers.html#sagemaker.serializers.CSVSerializer) object to serialize data of various formats to a CSV-formatted string.

In [25]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer

ll_predictor.serializer = CSVSerializer()
ll_predictor.deserializer = CSVDeserializer()

Note that we can also pass these Serializers in the deploy method call like this

```python
ll_predictor = ll_estimator.deploy(
    initial_instance_count=1, 
    instance_type='ml.t2.medium',
    endpoint_name=endpoint_name,
    serializer = CSVSerializer(),
    deserializer = CSVDeserializer()
)
```

In [37]:
##
# call the endpoint for live inference
ll_predictor.predict(test_sample)

[['29.98671531677246']]

The prediction output tells us that this house will cost $29986.

When we called the `estimator.deploy()` it has returned us `sagemaker.Predictor` object. If we want to call endpoint from a separate code then we can use the same 'Predictor' object to call it. You can read more about this class here [sagemaker.predictor.Predictor](https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html#sagemaker.predictor.Predictor)

In [34]:
from sagemaker.predictor import Predictor

ll_predictor_2 = Predictor(
    endpoint_name, 
    session, # If not specified, then one is created using the default AWS configuration chain
    CSVSerializer(), 
    CSVDeserializer()
)

In [36]:
ll_predictor_2.predict(test_sample)

[['29.98671531677246']]

# Clean Up

Remember that the inference machine will remain in use until you take it down yourself. This is unlike training where provisioning and decommissioning of machines are done automatically. We have used `ml.t2.medium` for hosting our model which costs around `$0.056` per hour. This is not much but still, do clean up your environment once you are done experimenting to avoid unnecessary bills. 

You can remove the endpoint with a single call.

In [38]:
##
# ll_predictor_2 will work too

ll_predictor.delete_endpoint()

# Summary

In this post we have learned about the SageMaker workflow, and how to train and deploy machine learning models with a handful of APIs from SageMaker SDK.